In [1]:
import pandas as pd

df = pd.read_clipboard(sep='|')

In [2]:
# just from clipboard stuff
df = df.iloc[1:, 1:-1]
df['DecisionOrder'] = df.index
df.reset_index(drop=True, inplace=True)
df.columns = df.columns.str.strip()
df['MatchId'] = df['MatchId'].astype(int)
df['MapName'] = df['MapName'].str.strip()
df['DecisionTeamId'] = df['DecisionTeamId'].astype(int)
df['OtherTeamId'] = df['OtherTeamId'].astype(int)
df['Decision'] = df['Decision'].str.strip()

df.head()

,MatchId,MapName,DecisionTeamId,OtherTeamId,Decision,Created,Updated,DecisionOrder
0,1,de_dust2,2,1,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,1
1,1,de_inferno,1,2,Pick,2021-03-25 22:16:41,2021-03-25 22:16:41,2
2,1,de_mirage,1,2,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,3
3,1,de_nuke,2,1,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,4
4,1,de_overpass,2,1,Pick,2021-03-25 22:16:41,2021-03-25 22:16:41,5


In [3]:
df.sort_values(by=['MatchId', 'DecisionOrder'], ascending=True, inplace=True)

In [4]:
map_names = df['MapName'].unique().tolist()
map_names

['de_dust2',
 'de_inferno',
 'de_mirage',
 'de_nuke',
 'de_overpass',
 'de_train',
 'de_vertigo']

In [5]:
manip_df = pd.concat([df[['MatchId']], pd.get_dummies(df['MapName'])], axis=1)
rolling_df = (1 - manip_df.groupby('MatchId')[map_names].rolling(7, min_periods=0).sum().shift(1, fill_value=0))
rolling_df = rolling_df.astype(int)
rolling_df.reset_index(drop=True, inplace=True)
rolling_df.columns = [x + '_is_available' for x in rolling_df.columns]
rolling_df

,de_dust2_is_available,de_inferno_is_available,de_mirage_is_available,de_nuke_is_available,de_overpass_is_available,de_train_is_available,de_vertigo_is_available
0,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1
2,0,0,1,1,1,1,1
3,0,0,0,1,1,1,1
4,0,0,0,0,1,1,1
5,0,0,0,0,0,1,1
6,0,0,0,0,0,0,1


In [6]:
df = pd.concat([df, rolling_df], axis=1)

In [7]:
df.head()

,MatchId,MapName,DecisionTeamId,OtherTeamId,Decision,Created,Updated,DecisionOrder,de_dust2_is_available,de_inferno_is_available,de_mirage_is_available,de_nuke_is_available,de_overpass_is_available,de_train_is_available,de_vertigo_is_available
0,1,de_dust2,2,1,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,1,1,1,1,1,1,1,1
1,1,de_inferno,1,2,Pick,2021-03-25 22:16:41,2021-03-25 22:16:41,2,0,1,1,1,1,1,1
2,1,de_mirage,1,2,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,3,0,0,1,1,1,1,1
3,1,de_nuke,2,1,Remove,2021-03-25 22:16:41,2021-03-25 22:16:41,4,0,0,0,1,1,1,1
4,1,de_overpass,2,1,Pick,2021-03-25 22:16:41,2021-03-25 22:16:41,5,0,0,0,0,1,1,1


In [8]:
# copy paste-able

def get_available_maps(df):
    df.sort_values(by=['MatchId', 'DecisionOrder'], ascending=True, inplace=True)
    map_names = df['MapName'].unique().tolist()

    # get which map we're talking about, OHE
    manip_df = pd.concat([df[['MatchId']], pd.get_dummies(df['MapName'])], axis=1)
    # 1 - what's not available
    rolling_df = (1 - (manip_df.groupby('MatchId')[map_names]
                               .rolling(7, min_periods=0)
                               .sum()  # rolling sum
                               .shift(1, fill_value=0)  # shift down so we know what's available
                               ))
    rolling_df = rolling_df.astype(int)
    rolling_df.reset_index(drop=True, inplace=True)
    rolling_df.columns = [x + '_is_available' for x in rolling_df.columns]
    rolling_df
    df = pd.concat([df, rolling_df], axis=1)
    return df